In [4]:
import snowflake.connector


conn = snowflake.connector.connect(
    user='shuckerow22',
    password='NEpatriots12!',
    account='sysdeyh-nhb69290',
    warehouse='my_first_warehouse',  # Specify the warehouse
    database='testdb',  # Specify the database
    schema='testschema'  # Specify the schema (can be created if doesn't exist)
    )


In [5]:
cs = conn.cursor()

In [4]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [7]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [8]:
cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")

ProgrammingError: 090105 (22000): Cannot perform CREATE SCHEMA. This session does not have a current database. Call 'USE DATABASE', or use a qualified name.

In [10]:
cs.execute("CREATE OR REPLACE TABLE test_table(col1 integer, col2 string)")
cs.execute("INSERT INTO test_table(col1, col2) VALUES (123, 'test string1'), (456, 'test string2')")

In [18]:
cs.execute('SELECT * FROM test_table')

In [19]:
import pandas as pd

rows = cs.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cs.description]

# Create a pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame (this automatically displays as a table in Jupyter or some IDEs)
print(df)

   COL1          COL2
0   123  test string1
1   456  test string2


In [ ]:
import pandas as pd
import os

# Directory containing your CSV files (same location as your script)
csv_directory = '.'  # Refers to the current directory

# Loop through each file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        # Load the CSV file
        file_path = os.path.join(csv_directory, filename)
        df = pd.read_csv(file_path)
        
        # Standardize the date format for 'OrderDate', 'ExpectedDeliveryDate', 'LastReceiptDate'
        df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce').dt.strftime('%m/%d/%Y')
        df['ExpectedDeliveryDate'] = pd.to_datetime(df['ExpectedDeliveryDate'], errors='coerce').dt.strftime('%m/%d/%Y')
        df['LastReceiptDate'] = pd.to_datetime(df['LastReceiptDate'], errors='coerce').dt.strftime('%m/%d/%Y')

        # Standardize the timestamp format for 'LastEditedWhen' and 'Right_LastEditedWhen'
        df['LastEditedWhen'] = pd.to_datetime(df['LastEditedWhen'], errors='coerce').dt.strftime('%m/%d/%Y %I:%M %p')
        df['Right_LastEditedWhen'] = pd.to_datetime(df['Right_LastEditedWhen'], errors='coerce').dt.strftime('%m/%d/%Y %I:%M %p')
        
        # Save the cleaned CSV back to the same location
        cleaned_file_path = os.path.join(csv_directory, 'cleaned_' + filename)
        df.to_csv(cleaned_file_path, index=False)
        print(f"Processed {filename}")


In [12]:
cs.execute("""
    CREATE OR REPLACE STAGE my_stage
    FILE_FORMAT = (TYPE = 'CSV')
""")

In [45]:
import os

# Directory where the cleaned CSV files are located
csv_directory = '.'  # Current directory

# Loop through and upload only files that start with 'cleaned_'
for filename in os.listdir(csv_directory):
    if filename.startswith('cleaned_cleaned_') and filename.endswith('.csv'):
        # Build the full file path
        full_file_path = os.path.join(csv_directory, filename)

        # Prepare the PUT command for each file
        upload_query = f"""
            PUT file://{full_file_path} @my_stage;
        """

        # Execute the query to upload the file
        cs.execute(upload_query)
        print(f"File {filename} uploaded successfully.")


File cleaned_cleaned_2019-8.csv uploaded successfully.
File cleaned_cleaned_2019-10.csv uploaded successfully.
File cleaned_cleaned_2021-6.csv uploaded successfully.
File cleaned_cleaned_2020-6.csv uploaded successfully.
File cleaned_cleaned_2021-10.csv uploaded successfully.
File cleaned_cleaned_2020-5.csv uploaded successfully.
File cleaned_cleaned_2019-5.csv uploaded successfully.
File cleaned_cleaned_2021-4.csv uploaded successfully.
File cleaned_cleaned_2021-9.csv uploaded successfully.
File cleaned_cleaned_2019-1.csv uploaded successfully.
File cleaned_cleaned_2019-6.csv uploaded successfully.
File cleaned_cleaned_2021-5.csv uploaded successfully.
File cleaned_cleaned_2020-8.csv uploaded successfully.
File cleaned_cleaned_2020-4.csv uploaded successfully.
File cleaned_cleaned_2021-3.csv uploaded successfully.
File cleaned_cleaned_2022-4.csv uploaded successfully.
File cleaned_cleaned_2022-3.csv uploaded successfully.
File cleaned_cleaned_2019-2.csv uploaded successfully.
File cle

In [64]:
create_table_query = """
    CREATE OR REPLACE TABLE purchase_orders (
        PurchaseOrderID INT,
        SupplierID INT,
        OrderDate DATE,
        DeliveryMethodID INT,
        ContactPersonID INT,
        ExpectedDeliveryDate DATE,
        SupplierReference STRING,
        IsOrderFinalized BOOLEAN,
        Comments STRING,
        InternalComments STRING,
        LastEditedBy INT,
        LastEditedWhen TIMESTAMP,
        PurchaseOrderLineID INT,
        StockItemID INT,
        OrderedOuters INT,
        Description STRING,
        ReceivedOuters INT,
        PackageTypeID INT,
        ExpectedUnitPricePerOuter FLOAT,
        LastReceiptDate DATE,
        IsOrderLineFinalized BOOLEAN,
        Right_LastEditedBy INT,
        Right_LastEditedWhen TIMESTAMP
    );
"""
cs.execute(create_table_query)
print("Table created successfully.")

Table created successfully.


In [13]:
# Command to remove all files from the Snowflake stage
remove_stage_query = "REMOVE @my_stage;"

# Execute the REMOVE command to clear the stage
cs.execute(remove_stage_query)
print("All files removed from the stage.")


All files removed from the stage.


In [65]:
load_data_query = """
    COPY INTO purchase_orders
    FROM @my_stage
    FILE_FORMAT = (
        TYPE = 'CSV',
        SKIP_HEADER = 1,
        DATE_FORMAT = 'MM/DD/YYYY',
        TIME_FORMAT = 'HH12:MI AM',
        TIMESTAMP_FORMAT = 'MM/DD/YYYY HH12:MI AM',
        FIELD_OPTIONALLY_ENCLOSED_BY = '"',  -- To handle commas and newlines inside quoted fields
        ESCAPE_UNENCLOSED_FIELD = None       -- To handle special characters
    );
"""
cs.execute(load_data_query)
print("Data loaded into table successfully.")

Data loaded into table successfully.


In [52]:
cs.execute("SELECT * FROM purchase_orders LIMIT 10")

rows = cs.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cs.description]

# Create a pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame (this automatically displays as a table in Jupyter or some IDEs)
print(df)

   PURCHASEORDERID  SUPPLIERID   ORDERDATE  DELIVERYMETHODID  CONTACTPERSONID  \
0                1           2  2019-01-01                 9                2   
1                1           2  2019-01-01                 9                2   
2                1           2  2019-01-01                 9                2   
3               10          10  2019-01-02                 8                2   
4               11          12  2019-01-02                 7                2   
5               11          12  2019-01-02                 7                2   
6               11          12  2019-01-02                 7                2   
7               12           4  2019-01-03                 7                2   
8               12           4  2019-01-03                 7                2   
9               12           4  2019-01-03                 7                2   

  EXPECTEDDELIVERYDATE SUPPLIERREFERENCE  ISORDERFINALIZED COMMENTS  \
0           2019-01-15          B2084

In [66]:
cs.execute("ALTER TABLE purchase_orders DROP COLUMN Comments, InternalComments")


Create a calculated field that shows purchase order totals, i.e., for each order, sum the line-item amounts (defined as ReceivedOuters * ExpectedUnitPricePerOuter), and name this field POAmount

In [ ]:
cs.execute("ALTER TABLE purchase_orders ADD COLUMN POAmount FLOAT")

In [75]:
cs.execute("UPDATE purchase_orders SET POAmount = (ReceivedOuters * ExpectedUnitPricePerOuter)")

# Load the supplier invoice XML data (you will again first stage the data and then move it into a table)
a.	shred the data into a table (preferably in the COPY INTO process) where each row corresponds to a single invoice

b.	Make sure to examine the structure of the XML file and also try different functions such as GETXML, GET, PARSE_XML, FLATTEN, etc. 

c.	When building your query to shred the data, try to keep it as simple as possible at first and only attempt to extract a single element or only try a single SQL clause or function to see what it produces. 


In [136]:
cs.execute("PUT file://../Supplier_Transactions_XML.xml @my_stage AUTO_COMPRESS=TRUE;")

In [151]:
cs.execute("CREATE OR REPLACE TABLE supplier_transactions_raw (src VARIANT);")


In [152]:
a = """COPY INTO supplier_transactions_raw
FROM @my_stage
FILE_FORMAT = (TYPE = XML);
"""

cs.execute(a)

In [153]:
b = """SELECT src
FROM supplier_transactions_raw
LIMIT 5;
"""

cs.execute(b)

rows = cs.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cs.description]

# Create a pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame (this automatically displays as a table in Jupyter or some IDEs)
print(df)

                                                 SRC
0  <root>\n  <row>\n    <SupplierTransactionID>13...


In [157]:
c = """
WITH xml_data AS (
    SELECT raw_xml:root AS root_content
    FROM supplier_transactions_raw
)
SELECT
    XMLGET(f.value, 'SupplierID')::NUMBER AS SupplierID
FROM xml_data,
LATERAL FLATTEN(input => root_content.row) f;

"""

cs.execute(c)

rows = cs.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cs.description]

# Create a pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame (this automatically displays as a table in Jupyter or some IDEs)
print(df)


ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 8 at position 38 unexpected 'row'.
syntax error line 8 at position 37 unexpected '.'.

I was not able to parse the XML file using the COPY INTO command. 

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

# Step 1: Parse the XML file
tree = ET.parse('../Supplier_Transactions_XML.xml')  # Replace with your actual XML file path
root = tree.getroot()

# Step 2: Create an empty list to store the extracted data
data = []

# Step 3: Loop through each row in the XML and extract the data
for row in root.findall('row'):
    SupplierTransactionID = row.find('SupplierTransactionID').text
    SupplierID = row.find('SupplierID').text
    TransactionTypeID = row.find('TransactionTypeID').text
    PurchaseOrderID = row.find('PurchaseOrderID').text
    PaymentMethodID = row.find('PaymentMethodID').text
    SupplierInvoiceNumber = row.find('SupplierInvoiceNumber').text
    TransactionDate = row.find('TransactionDate').text
    AmountExcludingTax = row.find('AmountExcludingTax').text
    TaxAmount = row.find('TaxAmount').text
    TransactionAmount = row.find('TransactionAmount').text
    OutstandingBalance = row.find('OutstandingBalance').text
    FinalizationDate = row.find('FinalizationDate').text
    IsFinalized = row.find('IsFinalized').text
    LastEditedBy = row.find('LastEditedBy').text
    LastEditedWhen = row.find('LastEditedWhen').text

    # Append the extracted data to the list
    data.append({
        'SupplierTransactionID': SupplierTransactionID,
        'SupplierID': SupplierID,
        'TransactionTypeID': TransactionTypeID,
        'PurchaseOrderID': PurchaseOrderID,
        'PaymentMethodID': PaymentMethodID,
        'SupplierInvoiceNumber': SupplierInvoiceNumber,
        'TransactionDate': TransactionDate,
        'AmountExcludingTax': AmountExcludingTax,
        'TaxAmount': TaxAmount,
        'TransactionAmount': TransactionAmount,
        'OutstandingBalance': OutstandingBalance,
        'FinalizationDate': FinalizationDate,
        'IsFinalized': IsFinalized,
        'LastEditedBy': LastEditedBy,
        'LastEditedWhen': LastEditedWhen
    })

# Step 4: Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

# Step 5: Display the DataFrame
print(df)




     SupplierTransactionID SupplierID TransactionTypeID PurchaseOrderID  \
0                      134          2                 5               1   
1                      169          4                 5               2   
2                      186          5                 5               3   
3                      215          7                 5               4   
4                      224         10                 5               5   
...                    ...        ...               ...             ...   
2433                335510          4                 5            2070   
2434                335511          4                 7            None   
2435                335512          7                 7            None   
2436                335843          4                 5            2071   
2437                335847          7                 5            2072   

     PaymentMethodID SupplierInvoiceNumber TransactionDate AmountExcludingTax  \
0                 

In [6]:
df.to_csv('supplier_transactions.csv', index=False)

In [14]:
cs.execute("PUT file://supplier_transactions.csv @my_stage AUTO_COMPRESS=TRUE;")

In [15]:
create_query = """CREATE OR REPLACE TABLE supplier_transactions (
    SupplierTransactionID INT,
    SupplierID INT,
    TransactionTypeID INT,
    PurchaseOrderID FLOAT,
    PaymentMethodID INT,
    SupplierInvoiceNumber FLOAT,
    TransactionDate DATE,
    AmountExcludingTax FLOAT,
    TaxAmount FLOAT,
    TransactionAmount FLOAT,
    OutstandingBalance FLOAT,
    FinalizationDate DATE,
    IsFinalized BOOLEAN,
    LastEditedBy INT,
    LastEditedWhen TIMESTAMP
);
"""

cs.execute(create_query)

In [17]:
load_data_query = """
    COPY INTO supplier_transactions
    FROM @my_stage
    FILE_FORMAT = (
    TYPE = 'CSV',
    SKIP_HEADER = 1,
    DATE_FORMAT = 'YYYY-MM-DD',  -- Adjusted to match the date format in your CSV
    TIME_FORMAT = 'HH12:MI AM',
    TIMESTAMP_FORMAT = 'YYYY-MM-DD HH12:MI:SS.FF',  -- Adjusted timestamp format
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',  -- To handle commas and newlines inside quoted fields
    ESCAPE_UNENCLOSED_FIELD = None       -- To handle special characters
);
"""
cs.execute(load_data_query)

In [23]:
create_table = """CREATE OR REPLACE TABLE purchases_grouped AS
    SELECT PurchaseOrderID, SupplierID, SUM(POAmount) AS POAmount
    FROM purchase_orders
    GROUP BY PurchaseOrderID, SupplierID
;
"""

cs.execute(create_table)

In [28]:
create_table = """CREATE OR REPLACE TABLE purchases_invoices AS
    SELECT A.PurchaseOrderID, A.SupplierID, A.POAmount, B.AmountExcludingTax, B.TaxAmount, B.TransactionAmount
    FROM purchases_grouped A
    JOIN supplier_transactions B
    ON A.PurchaseOrderID = B.PurchaseOrderID
;
"""

cs.execute(create_table)

In [29]:
create_view = """CREATE VIEW purchases_orders_and_invoices AS
    SELECT *, (AmountExcludingTax - POAmount) AS "Invoiced vs. Quoted"
    FROM purchases_invoices
;
"""

cs.execute(create_view)

In [1]:
import psycopg2

# Step 1: Connect to PostgreSQL
conn = psycopg2.connect(
    host="127.0.0.1",
    port="8765",
    database="WestCoastImporters", 
    user="jovyan", 
    password="postgres"
)

# Step 2: Create a cursor object
cur = conn.cursor()


In [2]:
local_file_path = 'supplier_case_data.csv'

copy_sql = """
COPY supplier_case TO STDOUT WITH CSV HEADER DELIMITER ',';
"""

# Step 4: Open a file and execute the copy command, writing the output to a file
with open(local_file_path, 'w') as f_output:
    cur.copy_expert(copy_sql, f_output)

# Close the cursor and connection
cur.close()
conn.close()

print(f"Data exported to {local_file_path}")

Data exported to supplier_case_data.csv


In [6]:
# Command to remove all files from the Snowflake stage
remove_stage_query = "REMOVE @my_stage;"

# Execute the REMOVE command to clear the stage
cs.execute(remove_stage_query)
print("All files removed from the stage.")


All files removed from the stage.


In [7]:
cs.execute("PUT file://supplier_case_data.csv @my_stage AUTO_COMPRESS=TRUE;")

In [8]:
create_query = """CREATE OR REPLACE TABLE supplier_case (
    supplierid INT,
    suppliername STRING,
    suppliercategoryid INT,
    primarycontactpersonid INT,
    alternatecontactpersonid INT,
    deliverymethodid FLOAT,
    postalcityid INT,
    supplierreference STRING,
    bankaccountname STRING,
    bankaccountbranch STRING,
    bankaccountcode INT,
    bankaccountnumber INT,
    bankinternationalcode INT,
    paymentdays INT,
    internalcomments STRING,
    phonenumber STRING,
    faxnumber STRING,
    websiteurl STRING,
    deliveryaddressline1 STRING,
    deliveryaddressline2 STRING,
    deliverypostalcode INT,
    deliverylocation STRING,
    postaladdressline1 STRING,
    postaladdressline2 STRING,
    postalpostalcode INT,
    lasteditedby INT,
    validfrom TIME,  -- This might need to be adjusted to DATE if it's actual date-time data
    validto STRING     -- Adjust to appropriate format if needed
);
"""

cs.execute(create_query)

In [10]:
load_data_query = """
    COPY INTO supplier_case
    FROM @my_stage
    FILE_FORMAT = (
    TYPE = 'CSV',
    SKIP_HEADER = 1,
    DATE_FORMAT = 'YYYY-MM-DD',  -- Adjusted to match the date format in your CSV
    TIME_FORMAT = 'HH:MI.SS.FF',
    TIMESTAMP_FORMAT = 'YYYY-MM-DD HH12:MI:SS.FF',  -- Adjusted timestamp format
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',  -- To handle commas and newlines inside quoted fields
    ESCAPE_UNENCLOSED_FIELD = None       -- To handle special characters
);
"""
cs.execute(load_data_query)